In [1]:
import numpy as np
import tensorflow as tf

from scada_forecast.preprocess import read_humidity, read_temperature, read_scada
from scada_forecast.preprocess import merge_dataframes, add_calendar_features, prepare_data

from scada_forecast.model import create_model, train_model, inference

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
df_humidity = read_humidity('data/scada/DoAm.xlsx')
df_temperature = read_temperature('data/NhietDoQuaKhu.xlsx')
df_scada = read_scada('data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx')

In [3]:
df = merge_dataframes(df_scada, df_temperature, df_humidity)

In [4]:
df = add_calendar_features(df, use_lunar=True, use_holiday=False)

F:\Anaconda3\envs\new_sklearn\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
hour_steps = int(60 / 5)
df, features_dict, categorical_cols, numeric_cols, target_col = prepare_data(df, 
                                                                             hour_steps=hour_steps, 
                                                                             use_temp=True, 
                                                                             use_humidity=True, 
                                                                             use_lunar=True,
                                                                             use_holiday=False)

In [6]:
dtype_dict = {col: df[col].dtype for col in categorical_cols + numeric_cols}

In [7]:
categorical_unique_values_dict = {
    'Month': list(range(1, 12 + 1)),
    'DayOfYear': list(range(1, 365 + 1)),
    'DayOfWeek': list(range(7)),
    'Hour': list(range(24)),
    'LunarMonth': list(range(1, 12 + 1)),
    'LunarDayOfMonth': list(range(1, 30 + 1)),
    'HolidayCat': list(range(12))
}

In [8]:
n_val = 100 * 24 * hour_steps # 100 days
n_test = 100 * 24 * hour_steps # 100 days
input_width = hour_steps
chosen_features = (features_dict['calendar_features'] + features_dict['lag_features'] + 
                   features_dict['temperature_features'] + features_dict['humidity_features'])
ckpt_path = 'tmp/ckpt'

In [ ]:
train_model(df, input_width, n_val, n_test, hour_steps,
            dtype_dict, categorical_unique_values_dict, 
            chosen_features, categorical_cols, 
            numeric_cols, target_col, ckpt_path)

In [9]:
forecast_df = inference(df, input_width, n_test, hour_steps, ckpt_path,
                        dtype_dict, categorical_unique_values_dict, 
                        chosen_features, categorical_cols, numeric_cols, target_col)

F:\Anaconda3\envs\new_sklearn\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['LogLoad'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
E:\EVN\package\scada_forecast\model.py:254: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_df['Forecast'] = forecast
